In [2]:
%run /home/sommema4/git/Timepix/Timepix_library/Cluster.ipynb

def go_through_files(path, extension):
    temp = os.path.join(path, '*.' + extension)
    print(temp)
    files = glob.glob(temp)
    return files

def read_clog(filename):
    # Opens .clog file, slices data into frames and saves data to a matrix
    cluster_list = []
    with open(filename) as f:
        for line in f:
            if (line.find('Frame') != -1):
                frame = line
            else:
                line = line.replace('[', '').replace(']','').replace(',', '')
                words = line.split()
                x = list(map(int, words[0::3]))
                y = list(map(int, words[1::3]))
                energy = list(map(float, words[2::3]))
                cluster = list(zip(x, y, energy))
                if cluster != []:
                    cluster_list.append((frame, cluster))
    return cluster_list
    
def get_properties(timepix):
    # Determines the properties of chip (mass and width) based on the manufacture number
    if (timepix == "H08-W0276" or "I08-W0276"):
        mass = 2.3008477184E-04
        width = 300
    elif (timepix == "C08-W0276"):
        mass = 3.8347461973E-04
        width = 500
    else:
        mass = 0
        width = 0
    return mass, width

def to_dataframe(data, tokens):
    dic = {'frame': 0, 'unix': 0, 'shutter': 0, 'volume': 0,
           'height': 0, 'size': 0, 'x_max': 0, 'y_max': 0,
           'Tx': 0, 'Ty': 0, 'volume_2': 0, 'height_2': 0}
    lst = []
    for frame, cluster in data:
        x, y, energy = list(zip(*cluster))
        x = list(x)
        y = list(y)
        energy = list(energy)
        if ('frame' in tokens or 'unix' in tokens or 'shutter' in tokens):
            dic['frame'], dic['unix'], dic['shutter'] = frame_to_words(frame)
        if ('volume' in tokens or 'height' in tokens or 'size' in tokens or 'x_max' in tokens or 'y_max' in tokens):
            dic['volume'], dic['height'], dic['size'], dic['x_max'], dic['y_max'] = get_vhs(x, y, energy)
        if ('Tx' in tokens or 'Ty' in tokens):
            dic['Tx'], dic['Ty'] = centroid_weighted(x, y, energy)
        if ('volume_2' in tokens or 'height_2' in tokens):
            dic['volume_2'], dic['height_2'] = hot_pixel_excluded(energy)
            
        temp = [dic[i] for i in tokens]
        lst.append(tuple(temp))
    df = pd.DataFrame(lst, columns=tokens)
    return df

def pixelize(df, x, y):
    df_x = df[df['x_max']==x]
    pixel = df_x[df_x['y_max']==y]
    return pixel

def pixelize_to_hdf(df, path, calibration_matrice, distance, bias, attribute):
    for i in range(256):
        print('Row: ', i)
        df_x = df[df['x_max']==i]
        directory = os.path.join(path, 'row_' + str(i))
        if not os.path.exists(directory):
            os.makedirs(directory)
        for j in range(256):
            file = os.path.join(directory, str(i) + 'x' + str(j) + '.h5')
            pixel = df_x[df_x['y_max']==j]           
            a, b, c, t = get_perpixel_coeffs(i, j, calibration_matrice)
            pixel['height_TOT'] = decalibrate_pixel(pixel['height'], a, b, c, t)
            save_table_to_hdf(pixel, file, [a, b, c, t], distance, bias, attribute)
            
def save_table_to_hdf(df, file, calibration, distance, bias, attribute):
    [a, b, c, t] = calibration
    with h5py.File(file, 'a', libver='latest') as hdf:
        if 'A' not in hdf.attrs.keys():
            hdf.attrs['A'] = a
            hdf.attrs['B'] = b
            hdf.attrs['C'] = c
            hdf.attrs['T'] = t
        
        if distance == None:
            group = hdf.require_group('undefined_dist_data')
        else:
            group = hdf.require_group(str(distance))
        if attribute == None:
            name = str(bias) + '_V'
        else:
            name = str(bias) + '_V_' + attribute
        dset = group.create_dataset(name, data=df)
        dset.attrs['BIAS'] = bias
        if distance != None:
            dset.attrs['DISTANCE'] = distance
        if attribute != None:
            dset.attrs['ATTRIBUTE'] = attribute

def frame_to_words(string):
    data = string.split(' ')
    frame_i = data[1]
    shutter = data[3]
    data[2] = data[2].replace("(", "")
    unix_time = data[2].replace(",", "")
    return frame_i, unix_time, shutter

def read_csv(filename):
    return pd.read_csv(filename)

def perpixel_calibration_matrice(path):
    # Save matrice to memory
    clog_file = os.path.join(path, 'A.txt')
    try:
        A = pd.read_csv(clog_file, sep=' ', header=None)
    except:
        print('Calibration file ' + clog_file + ' could not be found.')
    A = np.array(A)
    A = A.transpose()
    clog_file = os.path.join(path, 'B.txt')
    
    try:
        B = pd.read_csv(clog_file, sep=' ', header=None)
    except:
        print('Calibration file ' + clog_file + ' could not be found.')
    B = np.array(A)
    B = A.transpose()
    clog_file = os.path.join(path, 'C.txt')
    
    try:
        C = pd.read_csv(clog_file, sep=' ', header=None)
    except:
        print('Calibration file ' + clog_file + ' could not be found.')
    C = np.array(A)
    C = A.transpose()
    clog_file = os.path.join(path, 'T.txt')
    
    try:
        T = pd.read_csv(clog_file, sep=' ', header=None)
    except:
        print('Calibration file ' + clog_file + ' could not be found.')
    T = np.array(A)
    T = A.transpose()
    
    return [A, B, C, T]

def get_perpixel_coeffs(x, y, matrice):
    [A, B, C, T] = matrice
    a = float(A[x][y])
    b = float(B[x][y])
    c = float(C[x][y])
    t = float(T[x][y])
    return a, b, c, t
    